# Team IPPS 

The API information can be accessed through this [link](https://data.cms.gov/Medicare-Inpatient/Inpatient-Prospective-Payment-System-IPPS-Provider/97k6-zzx3)

Objectives (This will expand from just data frame formation to include cleanup by Jessie):
1. Execute a successful pull of all of the data using the API provided into JSON
2. Transform the JSON format into a data frame


### Step 1: Installing sodapy, a python package, in jupyter

In [3]:
# Installing the package sodapy, important for retrieval using the API's detailed method
import sys
!{sys.executable} -m pip install sodapy

distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


### Step 2: Retrieving the dataset

In [4]:
# The API link
link = "https://data.cms.gov/resource/ehrv-m9r6.json"

# The app token
key = "oBbcgRhXZS4dqtTJVyz6zQujv"

# Code snippet for data retrieval using python, as provided by the API information page
import pandas as pd
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.cms.gov", key)

# Example authenticated client (needed for non-public datasets):
# client = Socrata(data.cms.gov,
#                  MyAppToken,
#                  userame="user@example.com",
#                  password="AFakePassword")

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("ehrv-m9r6", limit=163065)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

# Checking the head of the data frame
results_df.head(10)

,average_covered_charges,average_medicare_payments,average_medicare_payments_2,drg_definition,hospital_referral_region_description,provider_city,provider_id,provider_name,provider_state,provider_street_address,provider_zip_code,total_discharges
0,32963.07,5777.24,4763.73,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,AL - Dothan,DOTHAN,10001,SOUTHEAST ALABAMA MEDICAL CENTER,AL,1108 ROSS CLARK CIRCLE,36301,91
1,15131.85,5787.57,4976.71,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,AL - Birmingham,BOAZ,10005,MARSHALL MEDICAL CENTER SOUTH,AL,2505 U S HIGHWAY 431 NORTH,35957,14
2,37560.37,5434.95,4453.79,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,AL - Birmingham,FLORENCE,10006,ELIZA COFFEE MEMORIAL HOSPITAL,AL,205 MARENGO STREET,35631,24
3,13998.28,5417.56,4129.16,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,AL - Birmingham,BIRMINGHAM,10011,ST VINCENT'S EAST,AL,50 MEDICAL PARK EAST DRIVE,35235,25
4,31633.27,5658.33,4851.44,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,AL - Birmingham,ALABASTER,10016,SHELBY BAPTIST MEDICAL CENTER,AL,1000 FIRST STREET NORTH,35007,18
5,16920.79,6653.80,5374.14,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,AL - Montgomery,MONTGOMERY,10023,BAPTIST MEDICAL CENTER SOUTH,AL,2105 EAST SOUTH BOULEVARD,36116,67
6,11977.13,5834.74,4761.41,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,AL - Birmingham,OPELIKA,10029,EAST ALABAMA MEDICAL CENTER AND SNF,AL,2000 PEPPERELL PARKWAY,36801,51
7,35841.09,8031.12,5858.5,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,AL - Birmingham,BIRMINGHAM,10033,UNIVERSITY OF ALABAMA HOSPITAL,AL,619 SOUTH 19TH STREET,35233,32
8,28523.39,6113.38,5228.4,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,AL - Huntsville,HUNTSVILLE,10039,HUNTSVILLE HOSPITAL,AL,101 SIVLEY RD,35801,135
9,75233.38,5541.05,4386.94,039 - EXTRACRANIAL PROCEDURES W/O CC/MCC,AL - Birmingham,GADSDEN,10040,GADSDEN REGIONAL MEDICAL CENTER,AL,1007 GOODYEAR AVENUE,35903,34


In [40]:
##state vs provider count
results_df["total_discharges"] = results_df["total_discharges"].astype(int)
stateGB = results_df.groupby(["provider_state"])
stateGB_result = pd.DataFrame(stateGB["provider_id"].nunique())
stateGB_result.rename(columns={"provider_id": "Provider Count"}, inplace=True)

## state vs total discharges
stateGB_result["Sum of Total Discharges"] = stateGB["total_discharges"].sum()


## state discharges vs DRG count
stateGB_result["DRG Count"] = stateGB["drg_definition"].nunique()
stateGB_result

,Provider Count,Sum of Total Discharges,DRG Count
provider_state,,,
AK,9,6142,78
AL,93,142704,100
AR,45,86769,100
AZ,61,104604,100
CA,299,474979,100
CO,45,61320,100
CT,31,96258,100
DC,7,20307,100
DE,6,26753,100


In [34]:
##drg code count by state, sort by highest
resultGB = results_df.groupby(["provider_state","drg_definition"])
drgCtByState = pd.DataFrame(resultGB["provider_id"].count())
drgCtByState.rename(columns = {"provider_id": "Provider Count"}, inplace = True)
drgCtByState["Rank"]= drgCtByState["Provider Count"].rank(method='max')
drgCtByStateRN = drgCtByState.sort_values(by="Rank", ascending = False)
drgCtByStateRNnoMax = drgCtByStateRN.reset_index()
drgCtByStateRNMaxreset = drgCtByStateRN.groupby(["provider_state"]).max().reset_index()
##join to filter out the highest rank by state
drgCTByStateRNJOIN = pd.merge(drgCtByStateRNreset, drgCtByStateRNnoMax, on=["provider_state", "Provider Count", "Rank"])
drgCTByStateRNJOIN

,provider_state,drg_definition,Provider Count,Rank
0,CA,871 - SEPTICEMIA OR SEVERE SEPSIS W/O MV 96+ H...,273,5025.0
1,CA,292 - HEART FAILURE & SHOCK W CC,256,5024.0
2,TX,690 - KIDNEY & URINARY TRACT INFECTIONS W/O MCC,255,5023.0
3,CA,690 - KIDNEY & URINARY TRACT INFECTIONS W/O MCC,254,5022.0
4,CA,"392 - ESOPHAGITIS, GASTROENT & MISC DIGEST DIS...",248,5021.0
5,CA,194 - SIMPLE PNEUMONIA & PLEURISY W CC,248,5021.0
6,TX,194 - SIMPLE PNEUMONIA & PLEURISY W CC,247,5019.0
7,CA,872 - SEPTICEMIA OR SEVERE SEPSIS W/O MV 96+ H...,239,5018.0
8,TX,"641 - MISC DISORDERS OF NUTRITION,METABOLISM,F...",239,5018.0
9,CA,291 - HEART FAILURE & SHOCK W MCC,238,5016.0
